In [2]:
#Import arcpy and set workspace
import arcpy
from arcpy import env
env.workspace="H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb"
env.addOutputsToMap = True

In [3]:
#Gathering state names for each county 
arcpy.management.MakeFeatureLayer(
    "H:/UEP 235/Final_Project/Federal Codes/Gazetteer_National_GDB/Gazetteer_National_GDB.gdb/Topical_Gazetteer/FederalCodes",
    "FederalCodesCounties",
    "Census_Class_Code LIKE 'H%' OR Census_Class_Code='C7'"
)
arcpy.conversion.ExportFeatures(
    in_features="FederalCodesCounties",
    out_features="FederalCodes_Counties",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='feature_id "Feature_ID" true true false 4 Long 0 0,First,#,FederalCodesCounties,feature_id,-1,-1;feature_name "Feature_Name" true true false 120 Text 0 0,First,#,FederalCodesCounties,feature_name,0,120;feature_class "Feature_Class" true true false 50 Text 0 0,First,#,FederalCodesCounties,feature_class,0,50;state_name "State_Name" true true false 100 Text 0 0,First,#,FederalCodesCounties,state_name,0,100;state_numeric "State_Numeric" true true false 2 Text 0 0,First,#,FederalCodesCounties,state_numeric,0,2;county_name "County_Name" true true false 100 Text 0 0,First,#,FederalCodesCounties,county_name,0,100;county_numeric "County_Numeric" true true false 5 Text 0 0,First,#,FederalCodesCounties,county_numeric,0,5',
)
arcpy.management.CalculateField(
    "FederalCodes_Counties",
    "County_State_ID",
    "!state_numeric!!county_numeric!",
)

<Result 'FederalCodes_Counties'>

In [4]:
# Adding state names to county shapefile
arcpy.management.AddJoin(
    in_layer_or_view="AllCounties2022",
    in_field="GEOID",
    join_table="FederalCodes_Counties",
    join_field="County_State_ID",
    join_type="KEEP_COMMON",
    index_join_fields="INDEX_JOIN_FIELDS"
)
arcpy.management.CopyFeatures(
    "AllCounties2022_Layer",
    "H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb/Boundary_Polygons/AllCountiesWStates"
)

<Result 'H:\\UEP 235\\Final_Project\\EQIPAnalysis\\EQIPAnalysis.gdb\\Boundary_Polygons\\AllCountiesWStates'>

In [5]:
#Extract contiguous US counties
arcpy.analysis.Select(
    "AllCountiesWStates",
    "H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb/Boundary_Polygons/ContiguousUSCounties",
    "AllCounties2022_STATEFP NOT IN ('11', '69', '60', '66', '15', '02', '72', '78','!',' ')"
)

<Result 'H:\\UEP 235\\Final_Project\\EQIPAnalysis\\EQIPAnalysis.gdb\\Boundary_Polygons\\ContiguousUSCounties'>

In [6]:
#Fix St. vs St issues
fc="ContiguousUSCounties_lyr"
fields=['AllCounties2022_NAME']

arcpy.management.MakeFeatureLayer(
    "ContiguousUSCounties",
    "ContiguousUSCounties_lyr"
)
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="ContiguousUSCounties_lyr",
    selection_type="NEW_SELECTION",
    where_clause="AllCounties2022_NAME LIKE 'St.%'",
)
with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        name=row[0]
        row[0] = name[0:2]+' '+name[4:]
        cursor.updateRow(row) 
        
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="ContiguousUSCounties_lyr",
    selection_type="CLEAR_SELECTION",
)

#Fix 's vs. s issues
fc="ContiguousUSCounties_lyr"
fields=['AllCounties2022_NAME']
arcpy.management.MakeFeatureLayer(
    "ContiguousUSCounties",
    "ContiguousUSCounties_lyr"
)
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="ContiguousUSCounties_lyr",
    selection_type="NEW_SELECTION",
    where_clause="AllCounties2022_NAME LIKE '%''s'",
)
with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        name=row[0]
        row[0] = name[:-2]+name[-1:]
        cursor.updateRow(row) 

arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="ContiguousUSCounties_lyr",
    selection_type="CLEAR_SELECTION",
)

<Result 'ContiguousUSCounties_lyr'>

In [7]:
#Create unique text ID for states and counties together
arcpy.management.CalculateField(
    in_table="ContiguousUSCounties",
    field="County_States",
    expression='Concatenate($feature.AllCounties2022_NAME,", ",$feature.FederalCodes_Counties_state_name)',
    expression_type="ARCADE",
    code_block="""# Joins two or more fields into a single string value
                # More calculator examples at esriurl.com/CalculatorExamples
                def ConcatenateFields(*args):
                sep = "-"
                return sep.join([i for i in args if i])""",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)
arcpy.management.CalculateField(
    "EQIPContracts",
    "County_States",
    expression='Concatenate($feature.County,", ",$feature.State)',
    expression_type="ARCADE",
    code_block="""# Joins two or more fields into a single string value
                # More calculator examples at esriurl.com/CalculatorExamples
                def ConcatenateFields(*args):
                sep = "-"
                return sep.join([i for i in args if i])""",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb\\EQIPContracts'>

In [35]:
#Join EQIP contract data onto US counties
arcpy.management.AddJoin(
    in_layer_or_view="ContiguousUSCounties_lyr",
    in_field="County_States",
    join_table="EQIPContracts",
    join_field="County_States",
    join_type="KEEP_COMMON",
    index_join_fields="INDEX_JOIN_FIELDS"
)
arcpy.management.CopyFeatures(
    "ContiguousUSCounties_lyr",
    "H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb/Boundary_Polygons/CountiesWContracts"
)
arcpy.management.RemoveJoin(
    "ContiguousUSCounties_lyr",
    "EQIPContracts"
)

<Result 'ContiguousUSCounties_lyr'>

In [36]:
#Create date field for space time cube
arcpy.management.ConvertTimeField(
    in_table="H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb/Boundary_Polygons/CountiesWContracts",
    input_time_field="EQIPContracts_FY",
    input_time_format="yyyy",
    output_time_field="FiscalYear",
    output_time_type="DATE",
)

<Result 'H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb/Boundary_Polygons/CountiesWContracts'>

In [38]:
#Filling in  suppressed values with zeros
arcpy.management.MakeFeatureLayer(
    "CountiesWContracts",
    "CountiesWContracts_lyr"
)
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="CountiesWContracts_lyr",
    selection_type="NEW_SELECTION",
    where_clause="EQIPContracts_Suppressed = 'TRUE'",
)
arcpy.management.CalculateField(
    "CountiesWContracts_lyr",
    "EQIPContracts_Contract_count",
    "0",
)
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="CountiesWContracts_lyr",
    selection_type="CLEAR_SELECTION",
)

<Result 'CountiesWContracts_lyr'>

In [40]:
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="CountiesWContracts_lyr",
    selection_type="NEW_SELECTION",
    where_clause="EQIPContracts_Suppressed = 'TRUE'",
)
arcpy.management.CalculateField(
    "CountiesWContracts_lyr",
    "EQIPContracts_Acres_Contracted",
    "0",
)
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="CountiesWContracts_lyr",
    selection_type="CLEAR_SELECTION",
)

<Result 'CountiesWContracts_lyr'>

In [42]:
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="CountiesWContracts_lyr",
    selection_type="NEW_SELECTION",
    where_clause="EQIPContracts_Suppressed = 'TRUE'",
)
arcpy.management.CalculateField(
    "CountiesWContracts_lyr",
    "EQIPContracts_Dollars_Obligated",
    "0",
)
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="CountiesWContracts_lyr",
    selection_type="CLEAR_SELECTION",
)

<Result 'CountiesWContracts_lyr'>

In [43]:
#Create space time cube
arcpy.stpm.CreateSpaceTimeCubeDefinedLocations(
    in_features="CountiesWContracts",
    output_cube="H:/UEP 235/Final_Project/EQIPAnalysis/EQIPContractsCube.nc",
    location_id="ContiguousUSCounties_AllCounties2022_GEOID",
    temporal_aggregation="NO_TEMPORAL_AGGREGATION",
    time_field="FiscalYear",
    time_step_interval="1 Years",
    time_step_alignment="END_TIME",
    reference_time=None,
    variables="EQIPContracts_Contract_count ZEROS;EQIPContracts_Acres_Contracted ZEROS;EQIPContracts_Dollars_Obligated ZEROS",
    summary_fields=None,
    in_related_table=None,
    related_location_id=None
)

<Result 'H:\\UEP 235\\Final_Project\\EQIPAnalysis\\EQIPContractsCube.nc'>

In [44]:
# Perform emerging hot spot analysis on each of the 3 variables of interest
arcpy.stpm.EmergingHotSpotAnalysis(
    "H:/UEP 235/Final_Project/EQIPAnalysis/EQIPContractsCube.nc",
    "EQIPCONTRACTS_CONTRACT_COUNT_NONE_ZEROS",
    "HotspotResults/Hotspots_contracts",
)
arcpy.stpm.EmergingHotSpotAnalysis(
    "H:/UEP 235/Final_Project/EQIPAnalysis/EQIPContractsCube.nc",
    "EQIPCONTRACTS_ACRES_CONTRACTED_NONE_ZEROS",
    "HotspotResults/Hotspots_acreage",
)
arcpy.stpm.EmergingHotSpotAnalysis(
    "H:/UEP 235/Final_Project/EQIPAnalysis/EQIPContractsCube.nc",
    "EQIPCONTRACTS_DOLLARS_OBLIGATED_NONE_ZEROS",
    "HotspotResults/Hotspots_dollars",
)

<Result 'H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb\\HotspotResults\\Hotspots_dollars'>

In [45]:
#Add state boundaries to map
arcpy.management.MakeFeatureLayer(
    "Boundary_Polygons/States",
    "States_lyr"
)

<Result 'States_lyr'>

In [46]:
#Mean Center Calculations for each variable over time
arcpy.stats.MeanCenter(
    Input_Feature_Class=r"H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\Boundary_Polygons\CountiesWContracts",
    Output_Feature_Class=r"H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\MeanCenters\Acreage",
    Weight_Field="EQIPContracts_Acres_Contracted",
    Case_Field="EQIPContracts_FY",
)
arcpy.stats.MeanCenter(
    Input_Feature_Class=r"H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\Boundary_Polygons\CountiesWContracts",
    Output_Feature_Class=r"H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\MeanCenters\Dollars",
    Weight_Field="EQIPContracts_Dollars_Obligated",
    Case_Field="EQIPContracts_FY",
)
arcpy.stats.MeanCenter(
    Input_Feature_Class=r"H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\Boundary_Polygons\CountiesWContracts",
    Output_Feature_Class=r"H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\MeanCenters\Contracts",
    Weight_Field="EQIPContracts_Contract_count",
    Case_Field="EQIPContracts_FY",
)

<Result 'H:\\UEP 235\\Final_Project\\EQIPAnalysis\\EQIPAnalysis.gdb\\MeanCenters\\Contracts'>

In [47]:
#Cleaning up county boundaries using a coastline file and clip
arcpy.analysis.Clip(
    "HotspotResults/Hotspots_contracts",
    "Boundary_Polygons/CountieswCoast",
    "HotspotResults/Hotspots_contracts_CLIP"
)
arcpy.analysis.Clip(
    "HotspotResults/Hotspots_acreage",
    "Boundary_Polygons/CountieswCoast",
    "HotspotResults/Hotspots_acreage_CLIP"
)
arcpy.analysis.Clip(
    "HotspotResults/Hotspots_dollars",
    "Boundary_Polygons/CountieswCoast",
    "HotspotResults/Hotspots_dollars_CLIP"
)

<Result 'H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb\\HotspotResults\\Hotspots_dollars_CLIP'>

In [48]:
#Combining cluster maps from contracts, dollars, and acreage into one map
arcpy.analysis.Intersect(
    in_features=r"'H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\HotspotResults\Hotspots_acreage_CLIP' #;'H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\HotspotResults\Hotspots_contracts_CLIP' #;'H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\HotspotResults\Hotspots_dollars_CLIP' #",
    out_feature_class=r"H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\HotspotResults\Hotspots_All",
    join_attributes="ALL"
)
arcpy.management.CalculateField(
    in_table="Hotspots_All",
    field="HC_1",
    expression="reclass(!CATEGORY!)",
    expression_type="PYTHON3",
    code_block="""def reclass(CATEGORY):
    if (CATEGORY > 2)and(CATEGORY < 7):
        return 1
    elif (CATEGORY < -2)and(CATEGORY > -7):
        return -1
    else:
        return 0""",
    field_type="DOUBLE",
)
arcpy.management.CalculateField(
    in_table="Hotspots_All",
    field="HC_2",
    expression="reclass(!CATEGORY_1!)",
    expression_type="PYTHON3",
    code_block="""def reclass(CATEGORY_1):
    if (CATEGORY_1 > 2)and(CATEGORY_1 < 7):
        return 1
    elif (CATEGORY_1 < -2)and(CATEGORY_1 > -7):
        return -1
    else:
        return 0""",
    field_type="DOUBLE",
)
arcpy.management.CalculateField(
    in_table="Hotspots_All",
    field="HC_3",
    expression="reclass(!CATEGORY_12!)",
    expression_type="PYTHON3",
    code_block="""def reclass(CATEGORY_12):
    if (CATEGORY_12 > 2)and(CATEGORY_12 < 7):
        return 1
    elif (CATEGORY_12 < -2)and(CATEGORY_12 > -7):
        return -1
    else:
        return 0""",
    field_type="DOUBLE",
)
arcpy.management.CalculateField(
    "Hotspots_All",
    "HC_Add",
    expression="reclass(!HC_1!+!HC_2!+!HC_3!)",
    expression_type="PYTHON3",
    code_block="""def reclass(sum):
    if (sum > 0):
        return 1
    elif (sum<0):
        return -1
    else:
        return 0""",
    field_type="DOUBLE",
)
arcpy.management.Dissolve(
    "Hotspots_All",
    "HotspotResults/AllDissolved",
    "HC_Add"
)

<Result 'H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb\\HotspotResults\\AllDissolved'>

In [54]:
#Creating farm resource regions map
arcpy.management.CalculateField(
    "FarmResourceRegions",
    "FIPS_Fixed",
    expression="reclass(!FIPS!)",
    expression_type="PYTHON3",
    code_block="""def reclass(FIPS):
    if (FIPS<10000):
        return "0"+str(FIPS)
    else:
        return FIPS""",
    field_type="TEXT",
)
arcpy.management.AddJoin(
    in_layer_or_view="ContiguousUSCounties_lyr",
    in_field="AllCounties2022_GEOID",
    join_table="FarmResourceRegions",
    join_field="FIPS_Fixed",
    join_type="KEEP_COMMON",
    index_join_fields="INDEX_JOIN_FIELDS"
)
arcpy.analysis.Clip(
    "ContiguousUSCounties_lyr",
    "Boundary_Polygons/CountieswCoast",
    "FarmResourceRegions/Counties"
)
arcpy.management.Dissolve(
    "FarmResourceRegions/Counties",
    "FarmResourceRegions/Regions",
    "FarmResourceRegions_Region"
)
arcpy.management.RemoveJoin(
    "ContiguousUSCounties_lyr",
    "FarmResourceRegions"
)

<Result 'ContiguousUSCounties_lyr'>

In [55]:
#Creating farm crop data map
arcpy.management.CalculateField(
    "crop_Farms",
    "ANSI_GEOID",
    expression="fix(!state_ANSI!,!County_ANSI!)",
    expression_type="PYTHON3",
    code_block="""def fix(state,county):
    if (state<10):
        if (county<10):
            return "0"+str(state)+"00"+str(county)
        elif (county>10)and(county<100):
            return "0"+str(state)+"0"+str(county)
        else:
            return "0"+str(state)+str(county)
    else:
        if (county<10):
            return str(state)+"00"+str(county)
        elif (county>10)and(county<100):
            return str(state)+"0"+str(county)
        else:
            return str(state)+str(county)""",
    field_type="TEXT",
)
arcpy.management.CalculateField(
    "total_Farms",
    "ANSI_GEOID",
    expression="fix(!state_ANSI!,!County_ANSI!)",
    expression_type="PYTHON3",
    code_block="""def fix(state,county):
    if (state<10):
        if (county<10):
            return "0"+str(state)+"00"+str(county)
        elif (county>10)and(county<100):
            return "0"+str(state)+"0"+str(county)
        else:
            return "0"+str(state)+str(county)
    else:
        if (county<10):
            return str(state)+"00"+str(county)
        elif (county>10)and(county<100):
            return str(state)+"0"+str(county)
        else:
            return str(state)+str(county)""",
    field_type="TEXT",
)
arcpy.management.AddJoin(
    in_layer_or_view="ContiguousUSCounties_lyr",
    in_field="AllCounties2022_GEOID",
    join_table="crop_Farms",
    join_field="ANSI_GEOID",
    join_type="KEEP_COMMON",
    index_join_fields="INDEX_JOIN_FIELDS"
)
arcpy.analysis.Clip(
    "ContiguousUSCounties_lyr",
    "Boundary_Polygons/CountieswCoast",
    "Boundary_Polygons/CountiesWCropData"
)
arcpy.management.RemoveJoin(
    "ContiguousUSCounties_lyr",
    "crop_Farms"
)

Traceback (most recent call last):
  File "<expression>", line 1, in <module>
  File "<string>", line 3, in fix
TypeError: '<' not supported between instances of 'NoneType' and 'int'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
  File "<string>", line 3, in fix
TypeError: '<' not supported between instances of 'NoneType' and 'int'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
  File "<string>", line 3, in fix
TypeError: '<' not supported between instances of 'NoneType' and 'int'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
  File "<string>", line 3, in fix
TypeError: '<' not supported between instances of 'NoneType' and 'int'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
  File "<string>", line 3, in fix
TypeError: '<' not supported between instances of 'NoneType' and 'int'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
  File 

<Result 'ContiguousUSCounties_lyr'>

In [56]:
#Creating farm crop data map (cont.)
arcpy.management.AddJoin(
    in_layer_or_view="CountiesWCropData",
    in_field="ContiguousUSCounties_AllCounties2022_GEOID",
    join_table="total_Farms",
    join_field="ANSI_GEOID",
    join_type="KEEP_COMMON",
    index_join_fields="INDEX_JOIN_FIELDS"
)
arcpy.analysis.Clip(
    "CountiesWCropData",
    "Boundary_Polygons/CountieswCoast",
    "Boundary_Polygons/CountiesWCropandFarmData"
)
arcpy.management.RemoveJoin(
    "CountiesWCropData",
    "total_Farms"
)
arcpy.management.CalculateField(
    "CountiesWCropandFarmData",
    "Percent_Crop",
    expression="!CountiesWCropData_crop_Farms_Crop_Farms!/!total_Farms_Total_Farms!*100",
    field_type="DOUBLE"
)

<Result 'CountiesWCropandFarmData'>

In [57]:
#Fixing ANSI codes in net cash farm income table and average farm size table
arcpy.management.CalculateField(
    "NCFI_Average",
    "ANSI_GEOID",
    expression="fix(!state_ANSI!,!county_ANSI!)",
    expression_type="PYTHON3",
    code_block="""def fix(state,county):
    if (state<10):
        if (county<10):
            return "0"+str(state)+"00"+str(county)
        elif (county>10)and(county<100):
            return "0"+str(state)+"0"+str(county)
        else:
            return "0"+str(state)+str(county)
    else:
        if (county<10):
            return str(state)+"00"+str(county)
        elif (county>10)and(county<100):
            return str(state)+"0"+str(county)
        else:
            return str(state)+str(county)""",
    field_type="TEXT",
)
arcpy.management.CalculateField(
    "Size_Average",
    "ANSI_GEOID",
    expression="fix(!state_ANSI!,!county_ANSI!)",
    expression_type="PYTHON3",
    code_block="""def fix(state,county):
    if (state<10):
        if (county<10):
            return "0"+str(state)+"00"+str(county)
        elif (county>10)and(county<100):
            return "0"+str(state)+"0"+str(county)
        else:
            return "0"+str(state)+str(county)
    else:
        if (county<10):
            return str(state)+"00"+str(county)
        elif (county>10)and(county<100):
            return str(state)+"0"+str(county)
        else:
            return str(state)+str(county)""",
    field_type="TEXT",
)

Traceback (most recent call last):
  File "<expression>", line 1, in <module>
  File "<string>", line 3, in fix
TypeError: '<' not supported between instances of 'NoneType' and 'int'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
  File "<string>", line 3, in fix
TypeError: '<' not supported between instances of 'NoneType' and 'int'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
  File "<string>", line 3, in fix
TypeError: '<' not supported between instances of 'NoneType' and 'int'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
  File "<string>", line 3, in fix
TypeError: '<' not supported between instances of 'NoneType' and 'int'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
  File "<string>", line 3, in fix
TypeError: '<' not supported between instances of 'NoneType' and 'int'
Traceback (most recent call last):
  File "<expression>", line 1, in <module>
  File 

<Result 'H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb\\Size_Average'>

In [58]:
#Adding farm data to hotspot data
arcpy.management.AddJoin(
    in_layer_or_view="Hotspots_All",
    in_field="LOC_LABEL",
    join_table="CountiesWCropandFarmData",
    join_field="CountiesWCropData_ContiguousUSCounties_AllCounties2022_GEOID",
    join_type="KEEP_COMMON",
    index_join_fields="INDEX_JOIN_FIELDS"
)
arcpy.analysis.Clip(
    "Hotspots_All",
    "Boundary_Polygons/CountieswCoast",
    "Boundary_Polygons/CountiesWCropandFarmData_Hotspots"
)

<Result 'H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb\\Boundary_Polygons\\CountiesWCropandFarmData_Hotspots'>

In [59]:
#Adding average farm size and average income to hotspot data
arcpy.management.AddJoin(
    in_layer_or_view="CountiesWCropandFarmData_Hotspots",
    in_field="Hotspots_All_LOC_LABEL",
    join_table="NCFI_Average",
    join_field="ANSI_GEOID",
    join_type="KEEP_COMMON",
    index_join_fields="INDEX_JOIN_FIELDS"
)
arcpy.analysis.Clip(
    "CountiesWCropandFarmData_Hotspots",
    "Boundary_Polygons/CountieswCoast",
    "Boundary_Polygons/CountiesWCropandFarmData_Hotspots_NCFI"
)
arcpy.management.RemoveJoin(
    "CountiesWCropandFarmData_Hotspots",
    "NCFI_Average"
)
arcpy.management.AddJoin(
    in_layer_or_view="CountiesWCropandFarmData_Hotspots_NCFI",
    in_field="CountiesWCropandFarmData_Hotspots_Hotspots_All_LOC_LABEL",
    join_table="Size_Average",
    join_field="ANSI_GEOID",
    join_type="KEEP_COMMON",
    index_join_fields="INDEX_JOIN_FIELDS"
)
arcpy.analysis.Clip(
    "CountiesWCropandFarmData_Hotspots_NCFI",
    "Boundary_Polygons/CountieswCoast",
    "Boundary_Polygons/CountiesWCropandFarmData_Hotspots_NCFIandSize"
)
arcpy.management.RemoveJoin(
    "CountiesWCropandFarmData_Hotspots_NCFI",
    "Size_Average"
)

<Result 'CountiesWCropandFarmData_Hotspots_NCFI'>

In [65]:
# Summary statistics for farm data by overall cluster type
arcpy.analysis.Statistics(
    in_table="CountiesWCropandFarmData_Hotspots_NCFIandSize",
    out_table=r"H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\Overall_Hotspot_Statistics",
    statistics_fields="Size_Average_Size_Avg MEAN;CountiesWCropandFarmData_Hotspots_NCFI_CountiesWCropandFarmDa_91 MEAN;CountiesWCropandFarmData_Hotspots_NCFI_NCFI_Average_NCFI_Avg MEAN;CountiesWCropandFarmData_Hotspots_NCFI_CountiesWCropandFarmDa_94 MEAN",
    case_field="CountiesWCropandFarmData_Hotspots_NCFI_CountiesWCropandFarmDa_50"
)

<Result 'H:\\UEP 235\\Final_Project\\EQIPAnalysis\\EQIPAnalysis.gdb\\Overall_Hotspot_Statistics'>

In [67]:
#Extracting overall hot and cold spot results by farm resource region
for r in range(1,10):
    arcpy.analysis.Select(
        "FarmResourceRegions/Regions",
        "FarmResourceRegions/Region"+str(r),
        where_clause="FarmResourceRegions_Region = "+str(r)
    )
    arcpy.analysis.Intersect(
        ["HotspotResults/Hotspots_All","FarmResourceRegions/Region"+str(r)],
        "HotspotsbyRegion/HotspotsRegion"+str(r)
    )

In [68]:
#Calculating perecent of counties in each region that are hot or cold
arcpy.analysis.Intersect(
    ["HotspotResults/Hotspots_All","FarmResourceRegions/Regions"],
    "HotspotsbyRegion/HotspotsRegion"
)
arcpy.analysis.Statistics(
    in_table=r"H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\HotspotsbyRegion\HotspotsRegion",
    out_table=r"H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\Counties_Stats_by_Region",
    statistics_fields="LOCATION COUNT",
    case_field="HC_Add;FarmResourceRegions_Region",
)
arcpy.conversion.ExportTable(
    in_table=r"H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\Counties_Stats_by_Region",
    out_table=r"H:\UEP 235\Final_Project\EQIPAnalysis\Counties_Stats_byReg.csv",
)

<Result 'H:\\UEP 235\\Final_Project\\EQIPAnalysis\\Counties_Stats_byReg.csv'>